# Sequential Deep Learning Models in Pytorch


In [2]:
#!/usr/bin/env python3
"""
@author Alberto Bassi & Riccardo Tomada
"""
# Essential
import numpy as np
import matplotlib.pyplot as plt
import os
import plotly.express as px
import pandas as pd

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets

# Neural Networks
from models import RNN_layer, LSTM_layer

# Progress bar
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
from time import sleep



## Dataset and Dataloader

In [19]:
# Visualize as pandas dataframe
file = pd.read_csv("bitcoin_dataset.csv",usecols = (1,2,3,4,5,6,7,8,9,10,11))
file

,Open Time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
0,2017-11-06 03:00:00.000000000,1.5000,1.7990,0.5000,1.7000,649.120,2017-11-06 03:59:59.999000064,7.251214e+02,33.0,207.450,3.514144e+02
1,2017-11-06 04:00:00.000000000,1.3000,1.6500,1.3000,1.6479,8147.720,2017-11-06 04:59:59.999000064,1.270853e+04,139.0,2130.590,3.436513e+03
2,2017-11-06 05:00:00.000000000,1.5457,1.5525,1.5455,1.5458,6628.200,2017-11-06 05:59:59.999000064,1.026534e+04,27.0,563.920,8.754265e+02
3,2017-11-06 06:00:00.000000000,1.5458,1.6810,1.5387,1.6810,22767.900,2017-11-06 06:59:59.999000064,3.650714e+04,133.0,12886.750,2.124768e+04
4,2017-11-06 07:00:00.000000000,1.6809,1.6809,1.6000,1.6250,14938.730,2017-11-06 07:59:59.999000064,2.427873e+04,58.0,7162.660,1.164245e+04
...,...,...,...,...,...,...,...,...,...,...,...
34746,2021-10-28 20:00:00,491.0000,494.8000,477.9000,490.5000,90431.134,2021-10-28 20:59:59.999000064,4.444162e+07,75832.0,40715.355,2.003381e+07
34747,2021-10-28 21:00:00,490.5000,490.5000,485.8000,489.8000,37248.759,2021-10-28 21:59:59.999000064,1.818572e+07,43707.0,19556.122,9.550490e+06
34748,2021-10-28 22:00:00,489.9000,491.3000,488.5000,489.3000,38162.017,2021-10-28 22:59:59.999000064,1.871114e+07,28600.0,21584.731,1.058593e+07
34749,2021-10-28 23:00:00,489.3000,492.3000,487.4000,491.9000,49811.734,2021-10-28 23:59:59.999000064,2.442102e+07,28422.0,25969.233,1.273187e+07


In [16]:
# Create a costum dataset class
class CsvDataset(Dataset):

    def __init__(self, csv_file, transform=None):
        """
        Args:
        csv_file (string): Path to the csv file.
        transform (callable, optional): Optional transform to be applied
            on a sample.
        """
        self.transform = transform
        
        # Read the file and split the lines in a list
        with open(csv_file, 'r') as f:
            lines = f.read().split('\n')
            
        # Get x and y values from each line and append to self.data
        self.data = []
        for line in lines:
            sample = line.split(',')
            len_sample = len(sample)
            self.data.append([sample[i] for i in range(len_sample)])
            # Now self.data contains all our dataset.
            # Each element of the list self.data is a tuple: (input, output)

    def __len__(self):
        # The length of the dataset is simply the length of the self.data list
        return len(self.data)

    def __getitem__(self, idx):
        # Our sample is the element idx of the list self.data
        sample = self.data[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample

## Transforms


In [17]:
class ToTensor(object):
    """Convert sample to Tensors."""

    def __call__(self, sample):
        x, y = sample
        return (torch.tensor([x]).float(),
                torch.tensor([y]).float())
    


In [18]:
# Dataset
composed_transform = transforms.Compose([ToTensor()])

dataset = CsvDataset('bitcoin_dataset.csv', transform=composed_transform)
print(dataset.__len__())
print(dataset.__getitem__(10))

34753


ValueError: too many values to unpack (expected 2)

In [ ]:
# Visualize dataset



## Dataloader

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
val_dataloader  = DataLoader(val_dataset,  batch_size=len(val_dataset), shuffle=False, num_workers=0)

## Define the Network

In [4]:
# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Training device: {device}")

### Set the random seed for reproducible results
torch.manual_seed(0)

### Initialize the two networks
x_dim = 3
h_dim = 3

net = LSTM_layer(x_dim, h_dim)

### Move to device
net.to(device)

Training device: cpu


LSTM_layer(
  (forget_gate): Linear(in_features=6, out_features=3, bias=True)
  (input_gate): Linear(in_features=6, out_features=3, bias=True)
  (cell_update): Linear(in_features=6, out_features=3, bias=True)
  (out): Linear(in_features=6, out_features=3, bias=True)
  (sigmoid): Sigmoid()
  (tanh): Tanh()
)

In [5]:
# Check if the nets are working properly

x = torch.randn(1,1,x_dim)
h = torch.randn(1,1,h_dim)
C = torch.randn(1,1,h_dim)

out = net(x,h, C)
print(out[0])

tensor([[[0.5053, 0.3733, 0.5591]]], grad_fn=<SigmoidBackward0>)


# Training

## Save and load model